In [ ]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
!pip install langchain_community
# !pip install transformers
# !pip install huggingface-hub



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# Load the free LLM from Hugging Face (flan-t5-small)
model_name = "google/flan-t5-small"

# Use Hugging Face pipeline with the selected model
hf_pipeline = pipeline(task="text2text-generation", model=model_name, max_new_tokens=100)


# Initialize the HuggingFacePipeline with the loaded pipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Now you can use `llm` in the same way as you would with AzureChatOpenAI

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import json
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.docstore.document import Document

In [ ]:
#Load the data
with open("sample_data/sample_causal.json", "r") as f:
    json_data = json.load(f)

# json_data

[{'sentence': 'A rise in unemployment leads to an increase in crime rates.',
  'cause': 'rise in unemployment',
  'effect': 'increase in crime rates'},
 {'sentence': 'Climate change is caused by deforestation and the burning of fossil fuels.',
  'cause': 'deforestation and burning of fossil fuels',
  'effect': 'climate change'},
 {'sentence': 'Political instability often results in economic downturns.',
  'cause': 'political instability',
  'effect': 'economic downturns'},
 {'sentence': 'High inflation reduces purchasing power and increases poverty.',
  'cause': 'high inflation',
  'effect': 'reduced purchasing power and increased poverty'},
 {'sentence': 'Inadequate healthcare infrastructure leads to higher mortality rates.',
  'cause': 'inadequate healthcare infrastructure',
  'effect': 'higher mortality rates'},
 {'sentence': 'A lack of education leads to higher unemployment rates.',
  'cause': 'lack of education',
  'effect': 'higher unemployment rates'},
 {'sentence': 'Trade imbal

In [ ]:
# 2. Create documents from the JSON data
docs = []
for entry in json_data:
    sentence = entry["sentence"]
    cause = entry["cause"]
    effect = entry["effect"]
    # Create Document objects for each entry
    docs.append(Document(page_content=sentence, metadata={"cause": cause, "effect": effect}))

# docs

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# 4. Split the documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'cause': 'rise in unemployment', 'effect': 'increase in crime rates'}, page_content='A rise in unemployment leads to an increase in crime rates.'),
 Document(metadata={'cause': 'deforestation and burning of fossil fuels', 'effect': 'climate change'}, page_content='Climate change is caused by deforestation and the burning of fossil fuels.'),
 Document(metadata={'cause': 'political instability', 'effect': 'economic downturns'}, page_content='Political instability often results in economic downturns.'),
 Document(metadata={'cause': 'high inflation', 'effect': 'reduced purchasing power and increased poverty'}, page_content='High inflation reduces purchasing power and increases poverty.'),
 Document(metadata={'cause': 'inadequate healthcare infrastructure', 'effect': 'higher mortality rates'}, page_content='Inadequate healthcare infrastructure leads to higher mortality rates.'),
 Document(metadata={'cause': 'lack of education', 'effect': 'higher unemployment rates'}, pag

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 5. Use Chroma to store the embeddings of the documents
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
# vectorstore


In [ ]:
# 6. Set up a retriever using the vectorstore
retriever = vectorstore.as_retriever()

In [ ]:
# 7. Load a free LLM from Hugging Face
model_name = "google/flan-t5-small"
hf_pipeline = pipeline(task="text2text-generation", model=model_name)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
# 8. Set up the RAG chain with a prompt for answering questions
prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces
of retrieved context to answer the question. If you don't know the answer,
just say that you don't know. Use three sentences maximum and keep the
answer concise.

Context: {context}

Question: {question}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Define a RAG chain using retrieval and LLM
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# 9. Test the chain by asking a question about the JSON data
response = rag_chain.invoke({"query": "What leads to bigger basketball courts?"})

# Extract the result (answer) from the response
result = response['result']
source_documents = response['source_documents']

# Print the result (the generated answer)
print(f"Answer: {result}")

# Optionally, print the source documents (retrieved chunks)
print("\nSource Documents:")
for doc in source_documents:
    print(f"- {doc.page_content}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Answer: A rise in unemployment

Source Documents:
- High inflation reduces purchasing power and increases poverty.
- High inflation reduces purchasing power and increases poverty.
- High inflation reduces purchasing power and increases poverty.
- A rise in unemployment leads to an increase in crime rates.


In [ ]:
# len(docs[0].page_content)
print(docs[0].page_content[:500])

A rise in unemployment leads to an increase in crime rates.
